# Part 3 - Analyze Data to Answer Questions

### Setting up the environment

In [1]:
# Import những thư viện cần thiết
import pandas as pd
import datetime
import re

### Reading the data 

In [2]:
# Đọc dữ liệu vào dataframe
data=pd.read_csv(r"C:\Users\VLN\Documents\Project_BusinessCase\edited_data_file.csv")
visitor_inf=pd.read_csv(r"C:\Users\VLN\Documents\Project_BusinessCase\visitor_information.csv")

## 1) Lượng khách hàng mới mua hàng theo ngày

### Mục đích: Tạo bảng first_time để đếm số đơn hàng theo từng ngày từ khách hàng mới

### 1.1) Xử lý file data

In [3]:
#Từ file data chỉ lấy thông tin những khách hàng đã mua hàng ít nhất 1 lần
data_oldcust=data.drop(labels=data.loc[data['Number_of_Orders']==0].index)
data_oldcust

,visitor_id,Number_of_Orders,create_date
237,/f4j8qx535UCFJNxx13w65G4Oio=,1,2019-11-01
276,/hYvcu32bHoLWxYT2XvPGvE7nWM=,1,2019-11-01
562,/w2K8geGTx+wX00VpxJiHeXfeJM=,1,2019-11-01
669,+2UZCewm6vsWZ51KA18TaQF5mIo=,1,2019-11-01
775,+Bs058M58ladlHd5UfAw6qiSQwo=,3,2019-11-01
...,...,...,...
3943633,ZPR2fSY3Syx0CEhnTHudbd+zWu0=,1,2020-02-08
3943698,zrcN/J8m8r3Nx/ZK+RKGDGPP6mk=,1,2020-02-08
3943710,ZRMYbrt66TGr3Ym45qAyna8vzdw=,2,2020-02-08
3943778,ZtRZJZTci1GqYL7vm9Kfs+bkH2M=,4,2020-02-08


In [4]:
# Sắp xếp visitor_id và create_date theo thứ tự tăng dần
data_oldcust=data_oldcust.sort_values(by=['visitor_id','create_date'],ascending=[True,True])

In [5]:
# Xoá thông tin các id 'Khách vãng lai' vì ta đang làm việc với các id có thông tin 
data_oldcust=data_oldcust.drop(labels=data.loc[data['visitor_id']=='Khách vãng lai'].index)
data_oldcust

,visitor_id,Number_of_Orders,create_date
2840311,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
947193,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
316154,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
434476,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
473916,++P59XK9q+K1acy8MeRVzfrfOz0=,2,2019-11-13
...,...,...,...
1735337,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14
1064873,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27
552156,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14
631036,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-16


In [6]:
#Lưu lại để dùng cho các câu sau
data_oldcust.to_csv(r"C:\Users\VLN\Documents\Project_BusinessCase\old_cust.csv",index=False)

In [7]:
# lấy thông tin lần đầu mua hàng ===> bỏ duplicate và giữ lại dòng đầu trong nhóm bị duplicate
first_time=data_oldcust.drop_duplicates(subset='visitor_id',keep='first')
first_time

,visitor_id,Number_of_Orders,create_date
2840311,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
947193,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
316154,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
434476,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
1302158,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04
...,...,...,...
315489,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08
1735337,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14
1064873,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27
552156,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14


In [8]:
# Sắp xếp index và sửa lại tên của cột cho phù hợp
first_time=first_time.sort_values(by=['visitor_id','create_date'],ascending=[True,True],ignore_index=True)
first_time=first_time.rename({'create_date':'first_buy'},axis=1)
first_time

,visitor_id,Number_of_Orders,first_buy
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04
...,...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14


### 1.2) Xử lý file visitor_information

In [9]:
visitor_inf.head()

,create_date,visitor_id,gender,age,visitor_source
0,2020-02-08,eLbvQsgdm1yEo44ggZA6i9mXYIg=,Nam,37.0,Cừa hàng
1,2020-02-08,5I6he68XtIQHpsRGKp0EsfyM3x4=,LGBT,0.0,Website công ty
2,2020-02-08,CP4VK05fHSxd8oMQ2TPPKp49ZtQ=,Nam,0.0,Website công ty
3,2020-02-08,k8lYxsZaAhgnQOgTTgFlVvw6GGQ=,Nữ,0.0,Website công ty
4,2020-02-08,x65MkmEJiaGCq3OGxuI2uvfq4os=,Không rõ,39.0,Sàn thương mại điện tử


In [10]:
# Đổi tiêu đề cột cho phù hợp
visitor_inf=visitor_inf.rename({'create_date':'first_visit'},axis=1)
visitor_inf

,first_visit,visitor_id,gender,age,visitor_source
0,2020-02-08,eLbvQsgdm1yEo44ggZA6i9mXYIg=,Nam,37.0,Cừa hàng
1,2020-02-08,5I6he68XtIQHpsRGKp0EsfyM3x4=,LGBT,0.0,Website công ty
2,2020-02-08,CP4VK05fHSxd8oMQ2TPPKp49ZtQ=,Nam,0.0,Website công ty
3,2020-02-08,k8lYxsZaAhgnQOgTTgFlVvw6GGQ=,Nữ,0.0,Website công ty
4,2020-02-08,x65MkmEJiaGCq3OGxuI2uvfq4os=,Không rõ,39.0,Sàn thương mại điện tử
...,...,...,...,...,...
126398,2020-02-04,UaPkbFX9VqV4szba0Ya/E5SFKS4=,Nam,35.0,Cừa hàng
126399,2020-02-05,y3cDfPPmSLJRtPGL0/pExUOAqeQ=,Không rõ,35.0,Sàn thương mại điện tử
126400,2020-02-06,snYIf8MqQkimpm2DpiTnXHx8IoM=,Nam,35.0,Cừa hàng
126401,2020-02-06,42YVqRkkg0q2PNI4UkLLdKY7SYs=,Nam,35.0,Cừa hàng


### 1.3) Merge 2 bảng first_time, visitor_inf và giữ lại các id ở bảng first_time

In [11]:
# Merge 2 bảng & giữ lại các id ở bảng first_time
first_time=pd.merge(first_time,visitor_inf,how='left',on='visitor_id')
first_time

,visitor_id,Number_of_Orders,first_buy,first_visit,gender,age,visitor_source
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12,2020-01-10,LGBT,21.0,Website công ty
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25,2019-11-25,Nữ,17.0,Website công ty
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09,2019-11-09,Nam,1.0,Website công ty
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04,2019-12-04,Không rõ,NaN,Website công ty
...,...,...,...,...,...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08,2019-11-01,Nữ,NaN,Website công ty
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14,2019-12-14,Không rõ,NaN,Website công ty
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27,2019-11-27,Không rõ,NaN,Website công ty
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14,2019-11-14,Không rõ,NaN,Website công ty


In [12]:
# Ta có thể coi bảng first_time như bảng information của các khách hàng
# Thay các giá trị 'NaN' ở cột 'age' thành 'Không rõ'
first_time['age']=first_time['age'].fillna('Không rõ')
first_time

,visitor_id,Number_of_Orders,first_buy,first_visit,gender,age,visitor_source
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12,2020-01-10,LGBT,21.0,Website công ty
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25,2019-11-25,Nữ,17.0,Website công ty
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09,2019-11-09,Nam,1.0,Website công ty
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04,2019-12-04,Không rõ,Không rõ,Website công ty
...,...,...,...,...,...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08,2019-11-01,Nữ,Không rõ,Website công ty
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14,2019-12-14,Không rõ,Không rõ,Website công ty
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27,2019-11-27,Không rõ,Không rõ,Website công ty
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14,2019-11-14,Không rõ,Không rõ,Website công ty


### 1.4) Lưu bảng first_time 

In [13]:
# Lưu lại bảng first_time dưới dạng 'csv' và đặt tên là customer_info
first_time.to_csv(r"C:\Users\VLN\Documents\Project_BusinessCase\customer_info.csv",index=False)

## 2) Lượng khách hàng cũ mua lại hàng theo ngày

### Mục đích: Tạo bảng not_first_time để đếm số đơn hàng theo từng ngày từ khách hàng cũ

In [14]:
data_oldcust.head()

,visitor_id,Number_of_Orders,create_date
2840311,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
947193,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
316154,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
434476,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
473916,++P59XK9q+K1acy8MeRVzfrfOz0=,2,2019-11-13


In [15]:
# lấy thông tin lần đầu mua hàng ===> bỏ duplicate và giữ lại dòng đầu trong nhóm bị duplicate
first_time=data_oldcust.drop_duplicates(subset='visitor_id',keep='first')
first_time

,visitor_id,Number_of_Orders,create_date
2840311,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
947193,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
316154,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
434476,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
1302158,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04
...,...,...,...
315489,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08
1735337,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14
1064873,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27
552156,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14


In [16]:
# Từ bảng data_oldcust, loại những index trong bảng first_time
not_first_time=data_oldcust.drop(labels=first_time.index)

# Sắp xếp lại bảng not_first_time tăng dần theo vistior_id & create_date
not_first_time.sort_values(by=['visitor_id','create_date'],ascending=[True,True],ignore_index=True,inplace=True)
not_first_time

,visitor_id,Number_of_Orders,create_date
0,++P59XK9q+K1acy8MeRVzfrfOz0=,2,2019-11-13
1,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-18
2,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-12-07
3,++Rc3Z/cGqApeiipO78fSuUXrdo=,2,2019-12-25
4,++Rc3Z/cGqApeiipO78fSuUXrdo=,2,2019-12-27
...,...,...,...
49540,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-22
49541,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-23
49542,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-30
49543,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-12-05


In [17]:
# Lưu lại bảng not_first_time dưới dạng 'csv' và đặt tên là not_first_time
not_first_time.to_csv(r"C:\Users\VLN\Documents\Project_BusinessCase\not_first_time.csv",index=False)

## 3) Tính tỷ lệ chuyển đổi ngay lần đầu khách hàng biết đến công ty

<strong> Công thức tính tỷ lệ chuyển đổi như sau:</strong> <br>
&nbsp;&nbsp;&nbsp;&nbsp;<mark>Conversion rate hằng ngày = lượng người mua hàng hằng ngày / lượng người ghé qua hằng ngày </mark><br>

Tuy nhiên, chỉ có dữ liệu về lần đầu 1 khách hàng ghé qua chứ không biết những ngày sau đó họ có ghé lại hay không nên ta điều chỉnh lại công thức trên như sau:<br>
&nbsp;&nbsp;&nbsp;&nbsp;<mark>Conversion rate when first visit hằng ngày = lượng người ghé qua lần đầu và mua hàng hằng ngày / lượng người ghé qua lần đầu hằng ngày<mark><br>


<strong>Để tính lượng người ghé qua lần đầu và mua hàng hằng ngày:</strong><br>
&nbsp;&nbsp;    Bước 1. Từ bảng customer_information, lấy ra những Id có first_buy và first_visit bằng nhau<br>
&nbsp;&nbsp;    Bước 2. Đếm lượng Id từng ngày và chia theo visitor_source <br>
Từ đó, ta có bảng first_visitBuy<br>

<strong>Để tính lượng người ghé qua lần đầu hàng ngày:</strong> <br>
&nbsp;&nbsp;    Từ bảng visitor_information, đếm lượng Id từng ngày và cũng chia theo visitor_source
Từ đó, ta có bảng first_visit

<strong>Bảng conversion_rate = bảng first_visitBuy merge với bảng first_visit</strong>


In [18]:
# Đọc dữ liệu
customer_info=pd.read_csv(r"C:\Users\VLN\Documents\Project_BusinessCase\customer_info.csv")
customer_info

,visitor_id,Number_of_Orders,first_buy,first_visit,gender,age,visitor_source
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12,2020-01-10,LGBT,21.0,Website công ty
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25,2019-11-25,Nữ,17.0,Website công ty
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09,2019-11-09,Nam,1.0,Website công ty
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04,2019-12-04,Không rõ,Không rõ,Website công ty
...,...,...,...,...,...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-11-08,2019-11-01,Nữ,Không rõ,Website công ty
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14,2019-12-14,Không rõ,Không rõ,Website công ty
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27,2019-11-27,Không rõ,Không rõ,Website công ty
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14,2019-11-14,Không rõ,Không rõ,Website công ty


In [19]:
#Truy vấn những id có first_buy = first_visit
first_visitBuy=customer_info.loc[customer_info['first_buy']==customer_info['first_visit']]
first_visitBuy

,visitor_id,Number_of_Orders,first_buy,first_visit,gender,age,visitor_source
1,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25,2019-11-25,Nữ,17.0,Website công ty
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09,2019-11-09,Nam,1.0,Website công ty
3,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,3,2019-12-04,2019-12-04,Không rõ,Không rõ,Website công ty
6,++eXAqAXiEeaVqRLmrOElOIgwi4=,1,2019-11-11,2019-11-11,Không rõ,Không rõ,Website công ty
...,...,...,...,...,...,...,...
39433,zz+3mUNMhQ10EBbMg/8WItiohU8=,4,2019-11-13,2019-11-13,Nữ,17.0,Website công ty
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14,2019-12-14,Không rõ,Không rõ,Website công ty
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27,2019-11-27,Không rõ,Không rõ,Website công ty
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14,2019-11-14,Không rõ,Không rõ,Website công ty


In [20]:
# Đếm lượng id từng ngày và chia theo visitor source
first_visitBuy=first_visitBuy.groupby(['first_buy','visitor_source']).count().reset_index()
first_visitBuy

,first_buy,visitor_source,visitor_id,Number_of_Orders,first_visit,gender,age
0,2019-11-01,Cừa hàng,50,50,50,50,50
1,2019-11-01,Sàn thương mại điện tử,16,16,16,16,16
2,2019-11-01,Website công ty,346,346,346,346,346
3,2019-11-02,Cừa hàng,52,52,52,52,52
4,2019-11-02,Sàn thương mại điện tử,17,17,17,17,17
...,...,...,...,...,...,...,...
291,2020-02-07,Sàn thương mại điện tử,3,3,3,3,3
292,2020-02-07,Website công ty,39,39,39,39,39
293,2020-02-08,Cừa hàng,18,18,18,18,18
294,2020-02-08,Sàn thương mại điện tử,9,9,9,9,9


In [21]:
# Xoá những cột ko cần thiết và sửa tiêu đề cột cho phù hợp
first_visitBuy=first_visitBuy.drop(labels=first_visitBuy.columns[3:],axis=1)
first_visitBuy

,first_buy,visitor_source,visitor_id
0,2019-11-01,Cừa hàng,50
1,2019-11-01,Sàn thương mại điện tử,16
2,2019-11-01,Website công ty,346
3,2019-11-02,Cừa hàng,52
4,2019-11-02,Sàn thương mại điện tử,17
...,...,...,...
291,2020-02-07,Sàn thương mại điện tử,3
292,2020-02-07,Website công ty,39
293,2020-02-08,Cừa hàng,18
294,2020-02-08,Sàn thương mại điện tử,9


In [22]:
# Đổi tên cột lại cho phù hợp
first_visitBuy=first_visitBuy.rename(columns={'first_buy':'Date','visitor_id':'first_visitBuy'})
first_visitBuy

,Date,visitor_source,first_visitBuy
0,2019-11-01,Cừa hàng,50
1,2019-11-01,Sàn thương mại điện tử,16
2,2019-11-01,Website công ty,346
3,2019-11-02,Cừa hàng,52
4,2019-11-02,Sàn thương mại điện tử,17
...,...,...,...
291,2020-02-07,Sàn thương mại điện tử,3
292,2020-02-07,Website công ty,39
293,2020-02-08,Cừa hàng,18
294,2020-02-08,Sàn thương mại điện tử,9


In [23]:
# # Load file visitor_information để tạo thành bảng first_visit
visitor_inf=pd.read_csv(r"C:\Users\VLN\Documents\Project_BusinessCase\visitor_information.csv")

In [24]:
# Đếm lượng id từng ngày và chia theo visitor source
first_visit=visitor_inf.groupby(['create_date','visitor_source']).count().reset_index()
# Xoá những cột ko cần thiết
first_visit=first_visit.drop(labels=['gender','age'],axis=1)

In [27]:
# Đổi tiêu đề cột lại cho phù hợp
first_visit=first_visit.rename({'visitor_id':'first_visit','create_date':'Date'},axis=1)
first_visit

,Date,visitor_source,first_visit
0,2019-11-01,Cừa hàng,170
1,2019-11-01,Sàn thương mại điện tử,56
2,2019-11-01,Website công ty,1972
3,2019-11-02,Cừa hàng,197
4,2019-11-02,Sàn thương mại điện tử,62
...,...,...,...
295,2020-02-07,Sàn thương mại điện tử,71
296,2020-02-07,Website công ty,180
297,2020-02-08,Cừa hàng,78
298,2020-02-08,Sàn thương mại điện tử,36


In [28]:
# Merge 2 bảng first_visitBuy & first_visit để tạo thành bảng conversion_rate
conversion_rate=pd.merge(first_visitBuy,first_visit,how="outer",on=['Date','visitor_source'])
conversion_rate

,Date,visitor_source,first_visitBuy,first_visit
0,2019-11-01,Cừa hàng,50.0,170
1,2019-11-01,Sàn thương mại điện tử,16.0,56
2,2019-11-01,Website công ty,346.0,1972
3,2019-11-02,Cừa hàng,52.0,197
4,2019-11-02,Sàn thương mại điện tử,17.0,62
...,...,...,...,...
295,2020-02-08,Website công ty,35.0,171
296,2019-12-14,Sàn thương mại điện tử,NaN,38
297,2020-01-26,Sàn thương mại điện tử,NaN,4
298,2020-01-27,Sàn thương mại điện tử,NaN,3


In [29]:
# Cột first_visitBuy có các giá trị NaN vì có những ngày khách hàng ghé thăm lần đầu tiên nhưng không ai vừa ghé 
# thăm lần đầu và vừa mua hàng nên ta thay những giá trị này là bằng 0 cho phù hợp hơn
conversion_rate['first_visitBuy']=conversion_rate['first_visitBuy'].fillna(0)
conversion_rate

,Date,visitor_source,first_visitBuy,first_visit
0,2019-11-01,Cừa hàng,50.0,170
1,2019-11-01,Sàn thương mại điện tử,16.0,56
2,2019-11-01,Website công ty,346.0,1972
3,2019-11-02,Cừa hàng,52.0,197
4,2019-11-02,Sàn thương mại điện tử,17.0,62
...,...,...,...,...
295,2020-02-08,Website công ty,35.0,171
296,2019-12-14,Sàn thương mại điện tử,0.0,38
297,2020-01-26,Sàn thương mại điện tử,0.0,4
298,2020-01-27,Sàn thương mại điện tử,0.0,3


In [30]:
# Lưu file
conversion_rate.to_csv(r"C:\Users\VLN\Documents\Project_BusinessCase\conversion_rate.csv",index=False)

## 4) Tính khoảng thời gian trung bình 1 người mua hàng lần nữa

<strong>Công thức:</strong><br>

&nbsp;&nbsp;&nbsp;&nbsp;Khoảng thời gian trung bình 1 người mua hàng lần nữa = (Thời gian lần cuối mua - Thời gian lần đầu mua) / Số lần mua

### Mục đích: Tạo bảng avg_nOdays_to_repurchase có các tham số trong công thức trên nhằm tính khoảng thời gian trung bình 1 người mua hàng lần nữa

In [31]:
# Bước 1. Tính số lần mua
# Từ bảng data_oldcust(file có Number_of_order >=1 ), đếm số lần mua của từng id
data_oldcust.head()

,visitor_id,Number_of_Orders,create_date
2840311,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
947193,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
316154,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
434476,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
473916,++P59XK9q+K1acy8MeRVzfrfOz0=,2,2019-11-13


In [32]:
# Xoá cột 'create_date' vì không cần thiết
df=data_oldcust.drop(labels='create_date',axis=1)
df

,visitor_id,Number_of_Orders
2840311,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2
947193,++HN809MR7t8V9QmdOEA36NubfI=,2
316154,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1
434476,++P59XK9q+K1acy8MeRVzfrfOz0=,1
473916,++P59XK9q+K1acy8MeRVzfrfOz0=,2
...,...,...
1735337,zzGUju7RRo8FrQOIOmNM9vgwuns=,1
1064873,zzURsuIJlSzPj3lOz19YWU1VMrE=,1
552156,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1
631036,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1


In [33]:
# Gom các id lại theo từng nhóm và đếm số lượng đơn hàng đã mua của từng id
df=df.groupby(['visitor_id']).count().reset_index()
df

,visitor_id,Number_of_Orders
0,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,1
1,++HN809MR7t8V9QmdOEA36NubfI=,1
2,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1
3,++P59XK9q+K1acy8MeRVzfrfOz0=,4
4,++RP+syfDvZiTf/Pmov3PdEeFWM=,1
...,...,...
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,6
39435,zzGUju7RRo8FrQOIOmNM9vgwuns=,1
39436,zzURsuIJlSzPj3lOz19YWU1VMrE=,1
39437,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,2


In [34]:
# Chuẩn bị dữ liệu cho bước 2
# Muốn tính khoảng thời gian trung bình để một id mua hàng 1 lần nữa nên ta cần xoá các id có số lần mua bằng 1 ra khỏi bảng df
df=df.drop(labels=df.loc[df['Number_of_Orders']==1].index)
df

,visitor_id,Number_of_Orders
3,++P59XK9q+K1acy8MeRVzfrfOz0=,4
5,++Rc3Z/cGqApeiipO78fSuUXrdo=,6
7,++oEHFOpZPhAsTkYw1CVb/+AYrU=,5
8,++sf+nIMstu6uI9e53II6GyjGQU=,2
10,++zohmaabt4uh3LNxdHGxOcdoT8=,2
...,...,...
39427,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,6
39431,zyxvPreQYyyQ8uBNWgY2jqBi1ew=,3
39433,zz+3mUNMhQ10EBbMg/8WItiohU8=,4
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,6


In [35]:
# Bước 2. Tính khoảng thời gian giữa lần đầu và lần cuối mà khách hàng mua
# delta = last_time - first_time
"""
Từ bảng not_first_time, bỏ đi duplicate của cột visitor_id và giữ lại dòng cuối cùng trong mỗi nhóm duplicate
( bảng này đã được sort theo cột create_date, ascending) như vậy là có được mốc last_time của 1 id
"""
last_time=not_first_time.drop_duplicates(subset='visitor_id', keep='last')
last_time

,visitor_id,Number_of_Orders,create_date
2,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-12-07
7,++Rc3Z/cGqApeiipO78fSuUXrdo=,1,2020-02-02
11,++oEHFOpZPhAsTkYw1CVb/+AYrU=,1,2019-12-01
12,++sf+nIMstu6uI9e53II6GyjGQU=,1,2019-12-05
13,++zohmaabt4uh3LNxdHGxOcdoT8=,1,2020-01-10
...,...,...,...
49533,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,1,2020-01-24
49535,zyxvPreQYyyQ8uBNWgY2jqBi1ew=,1,2020-01-09
49538,zz+3mUNMhQ10EBbMg/8WItiohU8=,1,2019-12-18
49543,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-12-05


In [36]:
# Đổi lại tiêu đề cột cho phù hợp 
last_time=last_time.rename({'create_date':'last_buy'},axis=1)
# Sort lại index của table last_time theo visitor_id
last_time=last_time.sort_values(by='visitor_id',ignore_index=True)
last_time

,visitor_id,Number_of_Orders,last_buy
0,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-12-07
1,++Rc3Z/cGqApeiipO78fSuUXrdo=,1,2020-02-02
2,++oEHFOpZPhAsTkYw1CVb/+AYrU=,1,2019-12-01
3,++sf+nIMstu6uI9e53II6GyjGQU=,1,2019-12-05
4,++zohmaabt4uh3LNxdHGxOcdoT8=,1,2020-01-10
...,...,...,...
15513,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,1,2020-01-24
15514,zyxvPreQYyyQ8uBNWgY2jqBi1ew=,1,2020-01-09
15515,zz+3mUNMhQ10EBbMg/8WItiohU8=,1,2019-12-18
15516,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-12-05


In [37]:
# Ghép 2 bảng last_time & customer_info lại trong đó lấy các visitor_id ở bảng last_time
last_time=pd.merge(last_time,customer_info,how='left',on='visitor_id')
last_time

,visitor_id,Number_of_Orders_x,last_buy,Number_of_Orders_y,first_buy,first_visit,gender,age,visitor_source
0,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-12-07,1,2019-11-12,2019-11-12,Nữ,19.0,Website công ty
1,++Rc3Z/cGqApeiipO78fSuUXrdo=,1,2020-02-02,1,2019-12-24,2019-12-23,Nữ,28.0,Cừa hàng
2,++oEHFOpZPhAsTkYw1CVb/+AYrU=,1,2019-12-01,2,2019-11-23,2019-11-23,Nữ,14.0,Website công ty
3,++sf+nIMstu6uI9e53II6GyjGQU=,1,2019-12-05,1,2019-12-02,2019-11-29,LGBT,18.0,Website công ty
4,++zohmaabt4uh3LNxdHGxOcdoT8=,1,2020-01-10,1,2019-11-17,2019-11-17,Nam,30.0,Website công ty
...,...,...,...,...,...,...,...,...,...
15513,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,1,2020-01-24,1,2019-11-20,2019-11-03,Nam,25.0,Website công ty
15514,zyxvPreQYyyQ8uBNWgY2jqBi1ew=,1,2020-01-09,1,2020-01-02,2020-01-02,Không rõ,Không rõ,Website công ty
15515,zz+3mUNMhQ10EBbMg/8WItiohU8=,1,2019-12-18,4,2019-11-13,2019-11-13,Nữ,17.0,Website công ty
15516,zz4YKQwMrdb6za9UWGnbxWHcLZY=,1,2019-12-05,1,2019-11-08,2019-11-01,Nữ,Không rõ,Website công ty


In [38]:
# Cuối cùng, tính khoảng thời gian delta cần tìm
temp=[]
for i in last_time.index:
    first=datetime.datetime.strptime(last_time.loc[i]['first_buy'],"%Y-%m-%d").date()
    last=datetime.datetime.strptime(last_time.loc[i]['last_buy'],"%Y-%m-%d").date()
    tdelta=last-first
    temp.append(int(re.findall('[0-9]+',str(tdelta))[0]))


In [39]:
df['tdelta']=temp
df

,visitor_id,Number_of_Orders,tdelta
3,++P59XK9q+K1acy8MeRVzfrfOz0=,4,25
5,++Rc3Z/cGqApeiipO78fSuUXrdo=,6,40
7,++oEHFOpZPhAsTkYw1CVb/+AYrU=,5,8
8,++sf+nIMstu6uI9e53II6GyjGQU=,2,3
10,++zohmaabt4uh3LNxdHGxOcdoT8=,2,54
...,...,...,...
39427,zyI5xVs7Av/37rWmlrJ3kIHzjkQ=,6,65
39431,zyxvPreQYyyQ8uBNWgY2jqBi1ew=,3,7
39433,zz+3mUNMhQ10EBbMg/8WItiohU8=,4,35
39434,zz4YKQwMrdb6za9UWGnbxWHcLZY=,6,27


In [40]:
# Lưu lại bảng df dưới dạng 'csv' tên là avg_nOdays_to_repurchase
df.to_csv(r"C:\Users\VLN\Documents\Project_BusinessCase\avg_nOdays_to_repurchase.csv",index=False)

## 5) Tính tỷ lệ giữ chân khách hàng trong 1 kì (period)

In [42]:
# Tạo biến period là tham số cho các khung thời gian khác nhau, chia làm 3 khung: 14, 30, và 50 ngày
# Chỉ cần thay đổi các tham số này để lấy được dữ liệu cần thiết cho từng kì khác nhau
period=14
#period=30
#period=50

In [43]:
data_oldcust

,visitor_id,Number_of_Orders,create_date
2840311,++DZ24GqVhNE0Bl7f3Zp06lWjj4=,2,2020-01-12
947193,++HN809MR7t8V9QmdOEA36NubfI=,2,2019-11-25
316154,++J4GEqZUrL8Rf7XrURsNNt6KPw=,1,2019-11-09
434476,++P59XK9q+K1acy8MeRVzfrfOz0=,1,2019-11-12
473916,++P59XK9q+K1acy8MeRVzfrfOz0=,2,2019-11-13
...,...,...,...
1735337,zzGUju7RRo8FrQOIOmNM9vgwuns=,1,2019-12-14
1064873,zzURsuIJlSzPj3lOz19YWU1VMrE=,1,2019-11-27
552156,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-14
631036,zzWdLg/Dyw9eyko6k+ZSDUuTio0=,1,2019-11-16


### 5.1) Ghép period vào bảng data_oldcust

In [44]:
df=data_oldcust.sort_values(by='create_date',ignore_index=True)
df

,visitor_id,Number_of_Orders,create_date
0,262uP5ixn2svd0vnlmZ8Asg0ckk=,1,2019-11-01
1,eKTU6a85IMYviAUneppyKDPwWJg=,1,2019-11-01
2,cLnneqkMyW0owR36QeSAyEFG9z4=,1,2019-11-01
3,MS36D4J+qmMhmrBWsr4yhVh3Bxk=,5,2019-11-01
4,KlTbKOBkNrppmmS9zO0fwduFKlw=,1,2019-11-01
...,...,...,...
88979,ahcCUOV0Y4Y9OsMBGYKjVH3hNzY=,1,2020-02-08
88980,U0x55gTkTC5cHeGCdSXm+xbFJkM=,1,2020-02-08
88981,qYu5dMtumvsNY5e0l+6b0acyEcI=,2,2020-02-08
88982,A+Jg0cZ57UP5CZy7Kc7Y0h/1/rE=,2,2020-02-08


In [45]:
x = datetime.date(2019,11,1)
y = datetime.date(2020,2,8)
y-x

datetime.timedelta(days=99)

In [46]:
value_of_column = []

interval_of_a_period = datetime.timedelta(days = period)
period_number = 1
first_date_in_period = datetime.datetime.strptime(df.loc[0]['create_date'], "%Y-%m-%d").date()

for i in df.index:
    if datetime.datetime.strptime(df.loc[i]['create_date'], "%Y-%m-%d").date() - first_date_in_period + datetime.timedelta(days = 1) > interval_of_a_period:
        first_date_in_period = datetime.datetime.strptime(df.loc[i]['create_date'], "%Y-%m-%d").date()
        period_number += 1
    if datetime.datetime.strptime(df.loc[len(df) - 1]['create_date'], "%Y-%m-%d").date() - first_date_in_period + datetime.timedelta(days = 1) < interval_of_a_period:
        break
    value_of_column.append('period' +'_'+ str(period_number))

for i in range(0, (len(df) - len(value_of_column)), 1):
    value_of_column.append("Not enough")

In [47]:
df['period'] = value_of_column
df

,visitor_id,Number_of_Orders,create_date,period
0,262uP5ixn2svd0vnlmZ8Asg0ckk=,1,2019-11-01,period_1
1,eKTU6a85IMYviAUneppyKDPwWJg=,1,2019-11-01,period_1
2,cLnneqkMyW0owR36QeSAyEFG9z4=,1,2019-11-01,period_1
3,MS36D4J+qmMhmrBWsr4yhVh3Bxk=,5,2019-11-01,period_1
4,KlTbKOBkNrppmmS9zO0fwduFKlw=,1,2019-11-01,period_1
...,...,...,...,...
88979,ahcCUOV0Y4Y9OsMBGYKjVH3hNzY=,1,2020-02-08,Not enough
88980,U0x55gTkTC5cHeGCdSXm+xbFJkM=,1,2020-02-08,Not enough
88981,qYu5dMtumvsNY5e0l+6b0acyEcI=,2,2020-02-08,Not enough
88982,A+Jg0cZ57UP5CZy7Kc7Y0h/1/rE=,2,2020-02-08,Not enough


## 5.2) Tính số khách ở đầu mỗi kì

In [48]:
df = df.drop_duplicates(subset= ['visitor_id', 'period'], keep= 'first')
df

,visitor_id,Number_of_Orders,create_date,period
0,262uP5ixn2svd0vnlmZ8Asg0ckk=,1,2019-11-01,period_1
1,eKTU6a85IMYviAUneppyKDPwWJg=,1,2019-11-01,period_1
2,cLnneqkMyW0owR36QeSAyEFG9z4=,1,2019-11-01,period_1
3,MS36D4J+qmMhmrBWsr4yhVh3Bxk=,5,2019-11-01,period_1
4,KlTbKOBkNrppmmS9zO0fwduFKlw=,1,2019-11-01,period_1
...,...,...,...,...
88977,w3q2i2BzRarTj6N8MGZKDq1nCqU=,1,2020-02-08,Not enough
88978,6T80SOh2a9OPWjmJPK00/rlxfIw=,2,2020-02-08,Not enough
88980,U0x55gTkTC5cHeGCdSXm+xbFJkM=,1,2020-02-08,Not enough
88981,qYu5dMtumvsNY5e0l+6b0acyEcI=,2,2020-02-08,Not enough


In [49]:
df=df.drop(labels=df[df['period']=="Not enough"].index)
df

,visitor_id,Number_of_Orders,create_date,period
0,262uP5ixn2svd0vnlmZ8Asg0ckk=,1,2019-11-01,period_1
1,eKTU6a85IMYviAUneppyKDPwWJg=,1,2019-11-01,period_1
2,cLnneqkMyW0owR36QeSAyEFG9z4=,1,2019-11-01,period_1
3,MS36D4J+qmMhmrBWsr4yhVh3Bxk=,5,2019-11-01,period_1
4,KlTbKOBkNrppmmS9zO0fwduFKlw=,1,2019-11-01,period_1
...,...,...,...,...
88059,klTBozlIOEe19DcyG8plKcqvFcw=,1,2020-02-06,period_7
88060,QrdwrFgeEA6ptKMHc5oUuCT0oRU=,7,2020-02-06,period_7
88062,hqxVwK5EtNmvtumWis6cY/cuPLU=,1,2020-02-06,period_7
88064,rkg9Ez9RtdKOFeokI+738zvVWSA=,2,2020-02-06,period_7


In [50]:
cust_retention_rate=df.groupby('period').count().reset_index()
cust_retention_rate=cust_retention_rate.drop(columns=cust_retention_rate.columns[2:])

In [51]:
list=[0]
for i in cust_retention_rate.index:
    if i>0:
        list.append(cust_retention_rate.loc[i-1]['visitor_id'])
cust_retention_rate['begin_of_period']=list
cust_retention_rate

,period,visitor_id,begin_of_period
0,period_1,8910,0
1,period_2,13122,8910
2,period_3,6256,13122
3,period_4,8955,6256
4,period_5,6423,8955
5,period_6,4870,6423
6,period_7,4884,4870


In [52]:
cust_retention_rate=cust_retention_rate.drop(columns='visitor_id')

## 5.3) Tính số khách mất đi trong mỗi kỳ

In [53]:
list = [0]
for i in range(1, len(df['period'].unique()) + 1):
    count = 0
    if i > 1:
        a = df.loc[df['period'] == 'period' +'_'+ str(i - 1)]['visitor_id'].unique()
        b = df.loc[df['period'] == 'period' +'_'+ str(i)]['visitor_id'].unique()
        for j in range(len(a)):
            if a[j] not in b:
                count += 1
        list.append(count)
cust_retention_rate['lost'] = list
cust_retention_rate

,period,begin_of_period,lost
0,period_1,0,0
1,period_2,8910,6925
2,period_3,13122,10806
3,period_4,6256,4816
4,period_5,8955,6631
5,period_6,6423,4869
6,period_7,4870,3123


## 5.4) Lưu file

In [54]:
cust_retention_rate.to_csv(r"C:\Users\VLN\Documents\Project_BusinessCase\cust_retention_rate_{period}.csv".format(period = period),index = False)